### Task 3 (LLM Chat Assistant with dynamic context based on query):

Design and implement a basic LLM based chatbot of your choice to dynamically get context via external sources (via api) and documents and answer the user query.

In [23]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install chromadb
!pip install accelerate
!pip install --upgrade accelerate
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


Download a PDF document for Chatbot QA

In [24]:
import requests


def download_file(url, local_filename):
    # Send a GET request to the URL
    response = requests.get(url)
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()
    # Open a local file with the specified name in binary write mode
    with open(local_filename, "wb") as f:
        # Write the content of the response to the local file
        f.write(response.content)
    print(f"File downloaded successfully as {local_filename}")


# Example usage
url = "https://gscs-b2c.lge.com/downloadFile?fileId=JVxsnYpsZC8KX5SryYGYA"  # Replace with your file URL
local_filename = "downloaded_file.pdf"  # Replace with your desired local file name

download_file(url, local_filename)

File downloaded successfully as downloaded_file.pdf


In [25]:
!pip install pypdf

Load the PDF document & read it

In [26]:
from langchain.document_loaders import PyPDFLoader

In [27]:
documents = ""
def load_documents(file):
    loader = PyPDFLoader(file)
    documents = loader.load()
    return documents

In [28]:
from langchain.docstore.document import Document


import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


from langchain.embeddings import HuggingFaceEmbeddings

In [29]:
documents = load_documents("./downloaded_file.pdf")
len(documents)
print(documents)

[Document(page_content="OWNER'S MANUAL\nWASHING MACHINE\nRead this owner's manual thor oughly before operating the \nappliance and keep it handy for reference at all times.\nENGLISH\nWM3600H*A\nMFL71693706\nRev.02_032323www.lg.com\nCopyright © 2020-2023 LG Electronics Inc. All Rights Reserved.Scan to see the online manual.", metadata={'source': './downloaded_file.pdf', 'page': 0}), Document(page_content='TABLE OF CONTENTS2\n3 IMPORTANT SAFETY \nINSTRUCTIONS\n3 READ ALL INSTRUCTIONS BEFORE USE\n3 WARNING STATEMENTS6 CAUTION STATEMENTS\n7 PRODUCT OVERVIEW\n7 Product Features\n10 INSTALLATION\n10 Before Installing11 Choosing the Proper Location\n12 Unpacking and Removi ng Shipping Material\n13 Connecting the Water Inlet Hoses\n14 Connecting the Drain Hose\n15 Leveling the Washer16 Final Installation Check\n18 OPERATION\n18 Before Use19 Sorting Laundry20 Loading the Washer\n21 Adding Cleaning Products\n24 Control Panel26 Wash Cycles\n29 Cycle Modifiers\n29 Options and Extra Functions\n32 S

we split the data into chunks of 1,000 characters, with an overlap
of 200 characters between the chunks, which helps to give better results
and contain the context of the information between chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [30]:

text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=40)
documents = text_splitter.split_documents(documents)

In [31]:
hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

Vector_db = Chroma.from_documents(
    collection_name="document_docs",
    documents=documents,
    embedding=hf_embed,
    persist_directory="./docs/Persist_dir",
)
Vector_db.similarity_search("dummy") 
Vector_db.persist()

Load the Embeddings DB

In [32]:
# Start here to load a previously-saved DB
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


vector_db_path = "./docs/Persist_dir"

hf_embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = Chroma(
    collection_name="document_docs",
    embedding_function=hf_embed,
    persist_directory=vector_db_path,
)

Let's test it with some sample question

In [33]:
def get_similar_docs(question, similar_doc_count):
    return Vector_db.similarity_search(question, k=similar_doc_count)


# Let's test it with some sample question:
for doc in get_similar_docs("My machine gives me noises?", 3):
    print(doc.page_content)

47 TROUBLESHOOTINGENGLISH
NoisesService resulting 
from improper 
installation is not covered.The warranty covers manufacturing defects only.
• Costs are incurred for:
Service trips to deliver, pick up, install the product or for instruction on product use. The 
removal and reinstallation of the product.
Banging noise when 
the washer is filling 
with water. (Water hammer)Household water supply pressure is very high.
• Adjust household water supply regulator to  a lower pressure or contact a plumber.
Problem Possible Cause & Solution
Clicking Door lock sounds when the door locks or unlocks.
• Normal operation
Spraying or hissing Water spraying or circulating during the cycle
• Normal operation
Humming or 
GurglingDrain pump is pumping water from the washer at times during a cycle.
• Normal operation
Water sloshing A liquid in the balance ring around  the pulsator helps the basket spin smoothly.
• Normal operation
Water being added 
after the washer has 
already begun to operateThe wash

Prompt engineering with langchain

Now will  compose with a language model and prompting strategy to make a langchain chain that answers questions

In [34]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction:
  You know about Indian Constitution provide the best answer for questions  .
  Use only information in the following paragraphs to answer the question at the end.
  Explain the answer with reference to these paragraphs.
  If you don't have the information in paragraph then give response "I dont't know".

  {context}

  Question: {question}

  Response:
  """

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain

In [36]:
def build_qa_chain():
    torch.cuda.empty_cache()
    model_name = "databricks/dolly-v2-3b"  # we can use dolly-v2-3b or dolly-v2-7b for smaller model and faster inferences.

    # Increase max_new_tokens for a longer response
    instruct_pipeline = pipeline(
        model=model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        return_full_text=True,
        max_new_tokens=256,
        top_p=0.95,
        top_k=50,
        model_kwargs={"load_in_8bit": True},
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True
    )

    prompt = PromptTemplate(input_variables=["context", "question"], template=template)

    hf_pipe = HuggingFacePipeline(pipeline=instruct_pipeline)
    # Set verbose=True to see the full prompt:
    return load_qa_chain(llm=hf_pipe, chain_type="stuff", prompt=prompt, verbose=False)

In [37]:
qa_chain = build_qa_chain()

Get similar documents for given user question and use as a context for Simple Question Answering(Chatbot system)

In [53]:
def displayHTML(html):
    """Display HTML in Jupyter notebook."""
    from IPython.display import HTML

    display(HTML(html))


def answer_question(question):
    similar_docs = get_similar_docs(question, similar_doc_count=3)
    result = qa_chain({"input_documents": similar_docs, "question": question})
    bot_answer = str(result['output_text'])
    if bot_answer.endswith("Response:"):
        bot_answer = bot_answer[:-len("Response:")]
    result_html = f"Question: {question}"
    result_html += "\n"
    result_html += f"Answer: {bot_answer}"
    result_html += ""
    displayHTML(result_html)

In [54]:
answer_question("my machine inlet filters are clogged. How i can resolve?")

/home/vanitha.alagarsamy/miniconda3/envs/langchain_test/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
